이번 프로젝트에서 쓰이는 모듈을 한 곳에 모았다.

In [7]:
import zipfile
import os
import glob

from PIL import Image

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import numpy as np

# 데이터 압축 풀기

훈련 및 테스트 데이터 압출파일을 한꺼번에 풀기 위한 함수를 구현했다.    
이미 데이터가 존재한다면 이 과정은 넘어가도 된다.

In [50]:
# 이미지 압출 푸는 소스
def unzip_file(root_path,foldername):
    path = root_path+foldername
    zip_file_names = [fn for fn in os.listdir(path) if fn.endswith('zip')]
    print(zip_file_names)
    count = 0
    for zip_name in zip_file_names:
        with zipfile.ZipFile(path+'/'+zip_name) as myzip:            
            image_list = myzip.infolist()
            for i, image in enumerate(image_list):
                image.filename = f'{foldername}_{i+count}'
                myzip.extract(image, path=root_path+"extracted/"+foldername)
            count += len(image_list)
            print(f'count{count}')
         

In [51]:
path = os.getenv('HOME')+'/aiffel/rock_scissor_paper_data/'
hand_type = ['rock', 'paper', 'scissor']
for foldername in hand_type:
    print(foldername)
    unzip_file(path,foldername)


rock
['rock.zip', 'rocks_complete.zip', 'rock(1).zip', 'rock(3).zip', 'rock(2).zip']
count100
count1205
count1305
count1405
count1505
paper
['papers_complete.zip', 'paper(2).zip', 'paper.zip', 'paper(1).zip', 'paper(3).zip']
count1108
count1208
count1308
count1408
count1508
scissor
['scissor(2).zip', 'scissor.zip', 'scissors.zip', 'scissors_complete.zip', 'scissor(1).zip']
count100
count200
count300
count1294
count1394


## 파일 경로 설정

In [8]:
root_path = os.getenv('HOME')+'/aiffel/rock_scissor_paper_data/'
extracted_path = root_path + "extracted/"
test_path = root_path + "test/"

# 데이터 준비

## 사이즈 전처리
사이즈를 28x28 크기로 조정해서 저장하는 부분

In [4]:
def resize_image(img_path):
    images=glob.glob(img_path+"/"+"*")  
    print(f"{len(images)} will be resized")
    
    target_size = (28,28)
    for img in images:
        old_image = Image.open(img)
        new_image = old_image.resize(target_size, Image.ANTIALIAS)
        new_image.save(img, "JPEG")
        
    print(f"{len(images)} image resized")

In [5]:
for foldername in os.listdir(extracted_path):
    print(foldername)
    resize_image(extracted_path+foldername)

rock
1505 will be resized
1505 image resized
paper
1508 will be resized
1508 image resized
scissor
1394 will be resized
1394 image resized


이미지가 원하는 형태로 resize 되었는지 확인

In [6]:
img = Image.open(extracted_path+'rock/rock_0')
img.size

(28, 28)

## 가위, 바위, 보 가져오는 함수

각 타입별 원하는 개수만큼 랜덤하게 이미지를 가져오게 처리했다.

In [15]:
""" 이미지 load
Args:
    img_root_path: 이미지 루트 경로
    number_of_data: 로드할 데이터 개수
Returns:
    이미지 데이터, 해당 이미지의 라벨
"""
def load_data(img_root_path, number_of_data = 300):
    
        """ 원하는 타입의 이미지 데이터를 원하는 만큼 랜덤하게 뽑아내는 generator
        Args:
            path: 이미지 가져올 위치
            hand_type: 분류 id -> {0:'scissor', 1:'rock', 2:'paper'}
            count_data: 랜덤하게 뽑을 개수
        Yield:
            이미지 데이터를 int32 타입의 ndarray로 반환
        """
        def get_image_generator(path, hand_type, count_data):
            _path = path+hand_type+'/*'
            result = np.array([])
            rng = np.random.default_rng()
            fileList = rng.choice(np.array(glob.glob(_path)), count_data, replace=False)
            for file in fileList:
                img = np.array(Image.open(file),dtype=np.int32)  
                yield img
    
    
        img_size=  28
        color=3
        # container 생성
        imgs = np.zeros(number_of_data * img_size * img_size * color, dtype=np.int32).reshape(number_of_data, img_size,img_size,color)
        labels = np.zeros(number_of_data, dtype=np.int32)
        # 가위 = 0, 바위 = 1, 보 = 2    
        count = 0
        # number_of_data가 3의 배수가 아니여서 쓰레기 값이 들어가는 것을 막기 위해서 나머지만큼 클래스 순서대로 채운다.
        quot, remainder = divmod(number_of_data, 3)
        hand_type = {0:'scissor', 1:'rock', 2:'paper'}
        for idx, _type in hand_type.items():
            # 나머지와 동일한 type의 개수를 나머지만큼 더한다.
            if idx == remainder:
                _quot = quot+remainder
            else:
                _quot = quot
                
            for img in get_image_generator(img_root_path, _type, _quot):
                imgs[count,:,:,:] = img
                labels[count] = idx
                count += 1

        return imgs, labels
        
        
""" 테스트 이미지 load
Args:
    number_of_data: 로드할 데이터 개수
Returns:
    이미지 데이터, 해당 이미지의 라벨
"""        
def load_test_data(number_of_data = 300):
    _path = test_path
    imgs, labels = load_data(_path, number_of_data)
    return imgs, labels

""" 훈련 이미지 load
Args:
    number_of_data: 로드할 데이터 개수
Returns:
    이미지 데이터, 해당 이미지의 라벨
""" 
def load_train_data(number_of_data = 300):
    _path = extracted_path
    imgs, labels = load_data(_path, number_of_data)
    return imgs, labels  

# 딥러닝 네트워크 설계하기

In [16]:
""" 이미지 딥러닝 네트워크 생성 함수
Args:
    n_conv_1 = 1번째 합성곱층 하이퍼파라미터
    n_conv_2 = 2번째 합성곱층 하이퍼파라미터
    n_conv_3 = 3번째 합성곱층 하이퍼파라미터
    n_dense = Dense 층 하이퍼파라미터
Returns:
    모델
""" 
def make_model(n_conv_1 = 16, n_conv_2 = 32, n_conv_3=32, n_dense = 32):
    model = keras.models.Sequential()
    # 입력부
    model.add(keras.layers.Conv2D(n_conv_1,(3,3), activation='relu', input_shape=(28,28,3), strides=1))
    model.add(keras.layers.MaxPool2D((2,2)))
    # 데이터 증강을 위해 randomrotation을 추가했다.
    model.add(keras.layers.experimental.preprocessing.RandomRotation(0.1))
    model.add(keras.layers.Conv2D(n_conv_2, (3,3), activation='relu'))
    model.add(keras.layers.MaxPool2D((2,2)))
    model.add(keras.layers.Conv2D(n_conv_3, (3,3), activation='relu'))
    model.add(keras.layers.MaxPool2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(n_dense, activation='relu'))
    # 과적합을 막기 위해 dropout을 추가했다.
    model.add(keras.layers.Dropout(0.1))
    # 출력부
    model.add(keras.layers.Dense(3, activation='softmax'))
    model.summary()
    return model

# 모델 훈련

#### 255를 나눠 정규화 시켜주는 이유

In [17]:
x_train, y_train = load_train_data(1000)
np.min(x_train), np.max(x_train)

(0, 255)

픽셀 하나의 값은 0\~255사이이다. 이것을 0\~1사이 값으로 나누기 위해 정규화를 시켰다.

In [54]:
model = make_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# 1000개씩 이미지를 가져오는 함수를 20번 실행했다.
# load_train_data가 랜덤하게 이미지를 가져오기 때문에 데이터가 완전 동일하진 않다.
image_count = 0
for _ in range(20):
    x_train, y_train = load_train_data(1000)
    image_count += x_train.shape[0]
    # 정규화
    x_train_norm = x_train/255.0
    model.fit(x_train_norm, y_train, epochs=15)
    test_loss, test_accuracy = model.evaluate(x_train_norm, y_train, verbose=2)
    print(test_loss, test_accuracy)
    
print(f'총 {image_count}개의 데이터가 로드되었다.')


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
random_rotation_3 (RandomRot (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 32)          9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 1, 1, 32)         

32/32 [==============================] - 0s 4ms/step - loss: 0.2773 - accuracy: 0.9020
Epoch 3/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2925 - accuracy: 0.8910
Epoch 4/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2928 - accuracy: 0.8830
Epoch 5/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2549 - accuracy: 0.9040
Epoch 6/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2333 - accuracy: 0.9140
Epoch 7/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2487 - accuracy: 0.9170
Epoch 8/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2555 - accuracy: 0.8960
Epoch 9/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2589 - accuracy: 0.9030
Epoch 10/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2428 - accuracy: 0.9140
Epoch 11/15
32/32 [==============================] - 0s 4ms/step - loss: 0.2284 - accuracy: 0.9220
Epoch 12/15
32/32 [==========

32/32 [==============================] - 0s 4ms/step - loss: 0.1341 - accuracy: 0.9500
Epoch 7/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1349 - accuracy: 0.9500
Epoch 8/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1368 - accuracy: 0.9530
Epoch 9/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1167 - accuracy: 0.9570
Epoch 10/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1638 - accuracy: 0.9350
Epoch 11/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1770 - accuracy: 0.9430
Epoch 12/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1290 - accuracy: 0.9480
Epoch 13/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1249 - accuracy: 0.9530
Epoch 14/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1116 - accuracy: 0.9570
Epoch 15/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1273 - accuracy: 0.9450
32/32 - 0s - loss: 0.0539

32/32 [==============================] - 0s 4ms/step - loss: 0.0821 - accuracy: 0.9730
Epoch 11/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1020 - accuracy: 0.9650
Epoch 12/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1797 - accuracy: 0.9400
Epoch 13/15
32/32 [==============================] - 0s 4ms/step - loss: 0.0722 - accuracy: 0.9740
Epoch 14/15
32/32 [==============================] - 0s 4ms/step - loss: 0.0660 - accuracy: 0.9780
Epoch 15/15
32/32 [==============================] - 0s 4ms/step - loss: 0.0560 - accuracy: 0.9820
32/32 - 0s - loss: 0.0405 - accuracy: 0.9840
0.040531400591135025 0.984000027179718
Epoch 1/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1533 - accuracy: 0.9420
Epoch 2/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1190 - accuracy: 0.9510
Epoch 3/15
32/32 [==============================] - 0s 4ms/step - loss: 0.1048 - accuracy: 0.9630
Epoch 4/15
32/32 [======================

32/32 [==============================] - 0s 4ms/step - loss: 0.0374 - accuracy: 0.9860
Epoch 15/15
32/32 [==============================] - 0s 4ms/step - loss: 0.0467 - accuracy: 0.9830
32/32 - 0s - loss: 0.0626 - accuracy: 0.9730
0.0626169964671135 0.9729999899864197
총 20000개의 데이터가 로드되었다.


# 모델 평가

평가를 10번 진행해 평균적으로 어느 정도의 정확도가 나오는지 파악해보았다.

In [55]:
accuracies = []
for _ in range(10):
    # 테스트 데이터 load
    x_test, y_test = load_test_data()
    # 정규화
    x_test_norm = x_test/255.0
    
    test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
    accuracies.append(test_accuracy)
    print(f'{_} : {test_loss}, {test_accuracy}')

10/10 - 0s - loss: 4.6923 - accuracy: 0.6333
0 : 4.692270755767822, 0.6333333253860474
10/10 - 0s - loss: 4.4533 - accuracy: 0.6633
1 : 4.453260898590088, 0.6633333563804626
10/10 - 0s - loss: 4.5104 - accuracy: 0.6333
2 : 4.510385513305664, 0.6333333253860474
10/10 - 0s - loss: 4.0670 - accuracy: 0.6600
3 : 4.067007064819336, 0.6600000262260437
10/10 - 0s - loss: 4.1694 - accuracy: 0.6567
4 : 4.169389724731445, 0.6566666960716248
10/10 - 0s - loss: 3.9706 - accuracy: 0.6533
5 : 3.9705638885498047, 0.653333306312561
10/10 - 0s - loss: 4.4941 - accuracy: 0.6300
6 : 4.494050979614258, 0.6299999952316284
10/10 - 0s - loss: 4.2109 - accuracy: 0.6533
7 : 4.210907936096191, 0.653333306312561
10/10 - 0s - loss: 4.4768 - accuracy: 0.6433
8 : 4.476759910583496, 0.6433333158493042
10/10 - 0s - loss: 4.8232 - accuracy: 0.6433
9 : 4.82315731048584, 0.6433333158493042


In [56]:
print(f'평균 정확도: {np.mean(accuracies)}')

평균 정확도: 0.6469999969005584


# 회고

* 폴더에서 파일을 읽어오는 것이 다양한 이미지가 아닌, 파일 저장된 순서대로 불러오기 때문에 데이터를 로드할 때 다양성이 떨어진다는 생각을 했다    
따라서 이미지를 로드할 때 폴더 내 파일을 랜덤하게 가져오는 방식을 구현해 적은 데이터지만 다양하게 불러오게 하였다.
* 이미지를 랜덤하게 가져오기 때문에 이미지 로드를 여러번 하며 더 많은 데이터를 학습시켰다.
* 성능을 높이기 위해서 이미지를 더 추가할 수도 있었지만, `RandomRotation`라는 데이터 증강 레이어를 추가했다.
* 적은 데이터의 특성으로 중복되는 데이터가 존재해 과적합이 일어날 가능성이 높다고 판단해 `Dropout` 레이어를 추가했다.
* 같은 모델이여도 테스트 데이터에 따라 정확도가 달라질 것이라 생각했고, 10번 평가한 것의 평균을 계산했다.